In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

data_path = "/content/drive/MyDrive/DS340_Final_Project/Audio_Song_Actors_01-24"



base_out = "/content/drive/MyDrive/DS340_Final_Project/song_augmented_spectrograms_speed"

os.makedirs(base_out, exist_ok=True)

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob

In [ ]:
speed_factors = [0.25, 0.4, 0.5, 0.8, 1.2, 1.5, 2, 2.5]

In [ ]:
emotion_map = {
    "01":"neutral","02":"calm","03":"happy","04":"sad",
    "05":"angry","06":"fearful","07":"disgust","08":"surprised"
}

In [ ]:
for rate in speed_factors:
    # 1) make folder for this speed
    speed_dir = os.path.join(base_out, f"speed_{rate}")
    os.makedirs(speed_dir, exist_ok=True)

    # 2) inside it, one subfolder per emotion
    for emo in emotion_map.values():
        os.makedirs(os.path.join(speed_dir, emo), exist_ok=True)

    # 3) process each actor/file
    for actor in tqdm(os.listdir(data_path), desc=f"Speed={rate}"):
        actor_path = os.path.join(data_path, actor)
        if not os.path.isdir(actor_path):
            continue

        for fn in os.listdir(actor_path):
            if not fn.lower().endswith(".wav"):
                continue

            # extract emotion code & lookup label
            code = fn.split("-")[2]
            emotion = emotion_map.get(code)
            if emotion is None:
                continue

            # load and time-stretch
            wav_path = os.path.join(actor_path, fn)
            y, sr    = librosa.load(wav_path, sr=None)
            y_mod    = librosa.effects.time_stretch(y, rate=rate)

            # mel-spectrogram → dB
            S    = librosa.feature.melspectrogram(y=y_mod, sr=sr)
            S_db = librosa.power_to_db(S, ref=np.max)

            # plot & save
            plt.figure(figsize=(3,3))
            librosa.display.specshow(S_db, sr=sr,
                                     x_axis='time', y_axis='mel')
            plt.axis('off')

            outname   = fn.replace(".wav", f"_speed{rate}.png")
            save_path = os.path.join(speed_dir, emotion, outname)
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close()

Speed=2.5: 100%|██████████| 24/24 [02:25<00:00,  6.05s/it]
